<!-- <div id="toc_container"> -->
<h2>Table of Contents</h2>
<ul class="toc_list">
  <a href="#1&nbsp;&nbsp;General-Analysis-Information">1&nbsp;&nbsp;General Analysis Information</a><br>
    <ul>
    <a href="#1.1&nbsp;&nbsp;Project-summary">1.1&nbsp;&nbsp;Project summary</a><br>
    <a href="#1.2&nbsp;&nbsp;Project-directory">1.2&nbsp;&nbsp;Project directory</a><br>
    </ul>
  <a href="#2&nbsp;&nbsp;Manifest-preparation">2&nbsp;&nbsp;Manifest preparation</a><br>
    <ul>
    <a href="#2.1&nbsp;&nbsp;Metadata-overview">2.1&nbsp;&nbsp;Metadata overview</a><br>
    <a href="#2.2&nbsp;&nbsp;Receipt-selection">2.2&nbsp;&nbsp;Receipt selection</a><br>
    <a href="#2.3&nbsp;&nbsp;Output-manifest">2.3&nbsp;&nbsp;Output manifest</a><br>  
    </ul>
  <a href="#3&nbsp;&nbsp;Sequencing-information">3&nbsp;&nbsp;Sequencing information</a><br>
    <ul>
    <a href="#3.1&nbsp;&nbsp;Yaml-parameters">3.1&nbsp;&nbsp;Yaml parameters</a><br>
    <a href="#3.2&nbsp;&nbsp;Pipeline-run">3.2&nbsp;&nbsp;Pipeline run</a><br>
    </ul>
  <a href="#4&nbsp;&nbsp;Quality-control">4&nbsp;&nbsp;Quality control</a><br>
    <ul>
    <a href="#4.1&nbsp;&nbsp;Trimming">4.1&nbsp;&nbsp;Trimming</a><br>
    <a href="#4.2&nbsp;&nbsp;Non-bacterial-reads">4.2&nbsp;&nbsp;Non-bacterial reads</a><br>
    <a href="#4.3&nbsp;&nbsp;Sequencing-depth-distribution-per-flow-cell">4.3&nbsp;&nbsp;Sequencing depth distribution per flow cell</a><br>
    <a href="#4.4&nbsp;&nbsp;Read-counts-after-pre-process-filtering">4.4&nbsp;&nbsp;Read counts after pre process filtering</a><br>
    </ul>
  <a href="#5&nbsp;&nbsp;Sample-selection">5&nbsp;&nbsp;Sample selection</a><br>
    <ul>
    <a href="#3.5&nbsp;&nbsp;Biological-replicates">3.5&nbsp;&nbsp;Biological replicates</a><br>
    <a href="#3.6&nbsp;&nbsp;QC-samples">3.6&nbsp;&nbsp;QC samples</a><br>
    </ul>
  <a href="#4&nbsp;&nbsp;Rarefaction-threshold">4&nbsp;&nbsp;Rarefaction threshold</a><br>
  <a href="#5&nbsp;&nbsp;Alpha-diversity">5&nbsp;&nbsp;Alpha diversity</a><br>
  <a href="#6&nbsp;&nbsp;Beta-diversity">6&nbsp;&nbsp;Beta diversity</a><br>
  <ul>
    <a href="#6.1&nbsp;&nbsp;Bray-Curtis">6.1&nbsp;&nbsp;Bray-Curtis</a><br>
    <a href="#6.2&nbsp;&nbsp;Jaccard">6.2&nbsp;&nbsp;Jaccard</a><br>
    <a href="#6.3&nbsp;&nbsp;Weighted-UniFrac">6.3&nbsp;&nbsp;Weighted UniFrac</a><br>
    <a href="#6.4&nbsp;&nbsp;Unweighted-UniFrac">6.4&nbsp;&nbsp;Unweighted UniFrac</a><br>
  </ul>
</ul>

In [1]:
%%bash
pip install git+https://github.com/webermarcolivier/statannot

  Cloning https://github.com/webermarcolivier/statannot to c:\users\slsevi~1\appdata\local\temp\pip-req-build-1qyzcyck
  Created wheel for statannot: filename=statannot-0.2.2-py3-none-any.whl size=11039 sha256=dfc70c601b2d8ee78052824536a1ebf48d0f8232164b654083c055aadebe7809
  Stored in directory: C:\Users\SLSEVI~1\AppData\Local\Temp\pip-ephem-wheel-cache-by_rppwz\wheels\48\70\5d\319e4b342e18b0080826e678683b28bb358cebe0c135651156
Successfully built statannot


  Running command git clone -q https://github.com/webermarcolivier/statannot 'C:\Users\SLSEVI~1\AppData\Local\Temp\pip-req-build-1qyzcyck'


In [2]:
from IPython.display import display
import os.path
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import glob
from scipy import stats
from itertools import combinations
#from skbio.stats.ordination import pcoa
#from skbio import DistanceMatrix
#import os; os.environ['R_HOME'] = "C:\\Miniconda\\lib\\R\\"
import rpy2
sns.set(style="whitegrid")
#import warnings
#warnings.filterwarnings('ignore')
from statannot import add_stat_annotation

<h2 id="1&nbsp;&nbsp;Ge">1&nbsp;&nbsp;General Analysis Information</h2>

<h3 id="1.1&nbsp;&nbsp;Project-summary">1.1&nbsp;&nbsp;Project summary</h3>

AIM 1 consists of a fecal extraction comparison using fresh-frozen fecal material from human donors, and artificial colonies. Samples were extracted using multiple methods, to be examined. The 16S rRNA gene was then sequenced using an Illumina MiSeq from three projects: NP0084-MB4, NP0084-MB5, NP0084-MB6. A summary of the three individual projects is included below:

- NP0084-MB4
- NP0084-MB5
- NP0084-MB6

<h3 id="1.2&nbsp;&nbsp;Project-summary">1.2&nbsp;&nbsp;Project directory</h3>

In [3]:
cgr_proj_dir='/DCEG/Projects/Microbiome/Analysis/NP0084_MB/20200410_2019.1'
proj_dir='/Users/slsevilla/Google Drive/MyDocuments_Current/Education/George Mason University/Dissertation/Data/Aim1/'
img_dir='/Users/slsevilla/Google Drive/MyDocuments_Current/Education/George Mason University/Dissertation/Data/Aim1/output/img'
%cd {proj_dir}

C:\Users\slsevilla\Google Drive\MyDocuments_Current\Education\George Mason University\Dissertation\Data\Aim1


<h2 id="2&nbsp;&nbsp;Manifest-preparation">2&nbsp;&nbsp;Manifest preparation</h3>

<h3 id="2.1&nbsp;&nbsp;Metadata-overview">2.1&nbsp;&nbsp;Metadata overview</h3>

Metadata for the project was included with the metagenomic pipeline requirements.

In [4]:
manifest_path = proj_dir+'manifest//raw_manifest.txt'

manifest = pd.read_csv(glob.glob(manifest_path)[0],sep='\t',index_col=0)
manifest = manifest.dropna(how='all', axis='columns')
manifest.columns = manifest.columns.str.replace(' ', '')

In [5]:
manifest.head()

,Source-PCR-Plate,Run-ID,Project-ID,CGR-Sample-ID,Sample-Type,Sample-Des,Subject-ID,Reciept,Ext-Company,Ext-Kit,Ext-Robotics,Homo-Status,Homo-Method,Homo-Holder
Sample ID,,,,,,,,,,,,,,
SC249358,PC04924_A_01,180112_M01354_0104_000000000-BFN3F,NP0084-MB4,SC249358,Ext_Control,art_col,DZ35322,sFEMB-001-R-002,Qiagen,DSP Virus,QIASymphony,Standard,Vertical,Tubes
SC249363,PC04924_F_01,180112_M01354_0104_000000000-BFN3F,NP0084-MB4,SC249363,Ext_Control,art_col,DZ35322,sFEMB-001-R-002,Qiagen,DSP Virus,QIASymphony,Standard,Vertical,Tubes
SC249366-PC04925-D-01,PC04925_D_01,180112_M03599_0134_000000000-BFD9Y,NP0084-MB4,SC249366,Ext_Control,Ext_Blank,Water,sFEMB-001-R-002,Qiagen,DSP Virus,QIASymphony,Standard,Vertical,Tubes
SC249366-PC07578-D-01,PC07578_D_01,180328_M01354_0106_000000000-BFMHC,NP0084-MB5,SC249366,Ext_Control,Ext_Blank,Water,sFEMB-001-R-002,Qiagen,DSP Virus,QIASymphony,Standard,Vertical,Tubes
SC249367,PC04924_A_02,180112_M01354_0104_000000000-BFN3F,NP0084-MB4,SC249367,Ext_Control,Ext_Blank,Water,sFEMB-001-R-002,Qiagen,DSP Virus,QIASymphony,Standard,Vertical,Tubes


In [6]:
#Print the variables for the metadata
for i in manifest.drop(columns=['Source-PCR-Plate','CGR-Sample-ID']).columns:
    print(i)
    print(*manifest[i].unique(), sep = ", ")  
    print('\n')

Run-ID
180112_M01354_0104_000000000-BFN3F, 180112_M03599_0134_000000000-BFD9Y, 180328_M01354_0106_000000000-BFMHC, 190617_M01354_0118_000000000-CHFG3


Project-ID
NP0084-MB4, NP0084-MB5, NP0084-MB6


Sample-Type
Ext_Control, Study, Seq_Control


Sample-Des
art_col, Ext_Blank, human_1, Robogut, ZymoBiomics, Biocollective, MSA, Seq_Blank


Subject-ID
DZ35322, Water, human_1, DZ35298, D6300, Biocollective, D6310, DZ35316, MSA1000, MSA1001, MSA1002, MSA1003, NTC_Blank, PCR_Blank, D6305, D6306, D6311


Reciept
sFEMB-001-R-002, sFEMB-001-R-003, sFEMB-001-R-004, sFEMB-001-R-005, sFEMB-001-R-006, sFEMB-001-R-007, sFEMB-001-R-008, sFEMB-001-R-009, sFEMB-001-R-010, sFEMB-001-R-011, sFEMB-001-R-012, sFEMB-001-R-013, sFEMB-001-R-014, sFEMB-001-R-015, sFEMB-001-R-016, sFEMB-001-R-017, sFEMB-001-R-034, sFEMB-001-R-037, sFEMB-001-R-038, sFEMB-001-R-039, sFEMB-001-R-040, sFEMB-001-R-041, sFEMB-001-R-042, sFEMB-001-R-043, sFEMB-001-R-044, sFEMB-001-R-045, sFEMB-001-R-046, sFEMB-001-R-047, sFEMB-001-R-0

In [7]:
manifest["Run-ID"].unique()

array(['180112_M01354_0104_000000000-BFN3F',
       '180112_M03599_0134_000000000-BFD9Y',
       '180328_M01354_0106_000000000-BFMHC',
       '190617_M01354_0118_000000000-CHFG3'], dtype=object)

In [8]:
#Rename run ids, remove well location
manifest.replace({"180112_M01354_0104_000000000-BFN3F":"Run1","180112_M03599_0134_000000000-BFD9Y":"Run2",
                 "180328_M01354_0106_000000000-BFMHC":"Run3","190617_M01354_0118_000000000-CHFG3":"Run4"},inplace=True)
manifest.replace({r'_._..': r''}, inplace=True, regex=True)
manifest.head()

,Source-PCR-Plate,Run-ID,Project-ID,CGR-Sample-ID,Sample-Type,Sample-Des,Subject-ID,Reciept,Ext-Company,Ext-Kit,Ext-Robotics,Homo-Status,Homo-Method,Homo-Holder
Sample ID,,,,,,,,,,,,,,
SC249358,PC04924,Run1,NP0084-MB4,SC249358,Ext_Control,art_col,DZ35322,sFEMB-001-R-002,Qiagen,DSP Virus,QIASymphony,Standard,Vertical,Tubes
SC249363,PC04924,Run1,NP0084-MB4,SC249363,Ext_Control,art_col,DZ35322,sFEMB-001-R-002,Qiagen,DSP Virus,QIASymphony,Standard,Vertical,Tubes
SC249366-PC04925-D-01,PC04925,Run2,NP0084-MB4,SC249366,Ext_Control,Ext_Blank,Water,sFEMB-001-R-002,Qiagen,DSP Virus,QIASymphony,Standard,Vertical,Tubes
SC249366-PC07578-D-01,PC07578,Run3,NP0084-MB5,SC249366,Ext_Control,Ext_Blank,Water,sFEMB-001-R-002,Qiagen,DSP Virus,QIASymphony,Standard,Vertical,Tubes
SC249367,PC04924,Run1,NP0084-MB4,SC249367,Ext_Control,Ext_Blank,Water,sFEMB-001-R-002,Qiagen,DSP Virus,QIASymphony,Standard,Vertical,Tubes


<h3 id="2.2&nbsp;&nbsp;Sample-selection">2.2&nbsp;&nbsp;Sample selection</h3>

There were a number of receipts included in this manifest, that will not be used. Additionally there is one sample type (Sample.Des=Biocollective) that will also not be used.

<h4 id="2.2.1&nbsp;&nbsp;Receipts-removed">2.2.1&nbsp;&nbsp;Receipts removed</h4>
Receipts were removed from the project for the following reasons:

- Receipts that failed sequencing: R-01*, R-04, R-012
- Receipts that were cancelled: R-23 to R-25*
- Receipts that contained residuals: R-13, R-14, R-17

*Not included in sequencing project, but found in LIMS

In [9]:
def sort_list (m,col):
    sort_list = [x for x in m[col].unique() if str(x) != 'nan']
    sort_list.sort()
    return sort_list

In [10]:
print ("Receipt", "Ext Kit", "Homogenization Method", sep = " - ")
for i in sort_list(manifest,'Reciept'):
    temp = manifest[(manifest['Reciept'] == i)]
    col2 = temp['Ext-Kit'].unique()
    col3 = temp['Homo-Status'].unique()
    print (i, col2[0], col3[0],sep=" - ")

Receipt - Ext Kit - Homogenization Method
sFEMB-001-R-002 - DSP Virus - Standard
sFEMB-001-R-003 - 96 MagBead DNA Extraction Kit - Standard
sFEMB-001-R-004 - 96 MagBead DNA Extraction Kit - Discard
sFEMB-001-R-005 - MagAttract PowerSoil DNA Kit - Standard
sFEMB-001-R-006 - MagAttract PowerSoil DNA Kit - Standard
sFEMB-001-R-007 - MagAttract PowerMicrobiome Kit - Standard
sFEMB-001-R-008 - MagAttract PowerMicrobiome Kit - Standard
sFEMB-001-R-009 - QIAamp with Modifications - Standard
sFEMB-001-R-010 - QIAamp with Modifications - Standard
sFEMB-001-R-011 - DSP Virus - Standard
sFEMB-001-R-012 - 96 MagBead DNA Extraction Kit - Discard
sFEMB-001-R-013 - MagAttract PowerSoil DNA Kit - Residual
sFEMB-001-R-014 - MagAttract PowerMicrobiome Kit - Residual
sFEMB-001-R-015 - 96 MagBead DNA Extraction Kit - Standard
sFEMB-001-R-016 - 96 MagBead DNA Extraction Kit - Standard
sFEMB-001-R-017 - 96 MagBead DNA Extraction Kit - Residual
sFEMB-001-R-034 - DNeasy PowerSoil Pro kit - Altered
sFEMB-001-R

In [11]:
m_rec = manifest.copy()

recp_num = ['04','12','13','14','17']
drop_recp = []

for i in recp_num:
    drop_recp.append("sFEMB-001-R-0" + i)

m_rec = manifest[~manifest.Reciept.isin(drop_recp)]
m_rec_rm = manifest[manifest.Reciept.isin(drop_recp)]
m_rec.shape

(424, 14)

<h4 id="2.2.2&nbsp;&nbsp;Samples-removed">2.2.2&nbsp;&nbsp;Samples removed</h4>

In [12]:
m_nonbio = m_rec[m_rec['Sample-Des']!="Biocollective"]
m_nonbio.shape

(389, 14)

In [13]:
def metacount_all(df_in):
    #Print the counts for all of the metadata
    m = df_in.drop(columns=['ExternalID','Project-ID','ExtractionBatchID'],errors='ignore')

    for i in m.columns:
        display(m[i].value_counts().rename_axis(i).to_frame('Number of samples'))

<h3 id="2.3&nbsp;&nbsp;Project-sample-metadata">2.3&nbsp;&nbsp;Project sample metadata</h3>


In [14]:
metacount_all(m_nonbio)

,Number of samples
Source-PCR-Plate,
PC04924,85
PC04925,79
PC22190,76
PC22192,75
PC07578,74


,Number of samples
Run-ID,
Run4,151
Run1,85
Run2,79
Run3,74


,Number of samples
CGR-Sample-ID,
SC249448,4
SC253846,3
SC253201,3
SC249408,3
SC249437,3
...,...
SC284732,1
SC552952,1
SC553082,1


,Number of samples
Sample-Type,
Study,180
Ext_Control,167
Seq_Control,42


,Number of samples
Sample-Des,
human_1,131
ZymoBiomics,91
Ext_Blank,58
Robogut,49
art_col,30
MSA,20
Seq_Blank,10


...

,Number of samples
Ext-Kit,
MagAttract PowerMicrobiome Kit,75
MagAttract PowerSoil DNA Kit,74
96 MagBead DNA Extraction Kit,59
DSP Virus,45
QIAamp with Modifications,39
DNeasy PowerSoil Pro kit,38
MagMax Microbiome Ultra Kit,17


,Number of samples
Ext-Robotics,
KingFisher,166
None,120
QIASymphony,45
QIACube HT,16


,Number of samples
Homo-Status,
Standard,269
Covaris,30
SPEX,24
Altered,16
Horizontal,8


,Number of samples
Homo-Method,
Vertical,139
TissueLyzer,128
AFA,30
SPEX,24
Plate Adaptor,18
Horizontal,8


,Number of samples
Homo-Holder,
Tubes,181
Plate,166


In [15]:
def metacount_run_sub(df_in):
    tmp = df_in[['Run-ID','Subject-ID']].copy()
    tmp = tmp.groupby(['Run-ID','Subject-ID'])
    display(tmp.size().to_frame('Subject-ID').join(tmp.apply(list).apply(pd.Series)))

In [16]:
metacount_run_sub(m_nonbio)

Subject-ID       0           1
Run-ID Subject-ID                                
Run1   D6300               14  Run-ID  Subject-ID
       D6305                1  Run-ID  Subject-ID
       D6306                1  Run-ID  Subject-ID
       DZ35298             14  Run-ID  Subject-ID
       DZ35322             14  Run-ID  Subject-ID
       MSA1000              1  Run-ID  Subject-ID
       MSA1001              1  Run-ID  Subject-ID
       MSA1002              1  Run-ID  Subject-ID
       MSA1003              1  Run-ID  Subject-ID
       NTC_Blank            1  Run-ID  Subject-ID
       PCR_Blank            1  Run-ID  Subject-ID
       Water                7  Run-ID  Subject-ID
       human_1             28  Run-ID  Subject-ID
Run2   D6300                8  Run-ID  Subject-ID
       D6305                1  Run-ID  Subject-ID
       D6306                1  Run-ID  Subject-ID
       DZ35298             12  Run-ID  Subject-ID
       DZ35322              4  Run-ID  Subject-ID
       MSA1000              1  Run-ID  Subject-ID
       MSA1001              1  Run-ID  Subject-ID
       MSA1002              1  Run-ID  Subject-ID
       MSA1003              1  Run-ID  Subject-ID
       NTC_Blank            1  Run-ID  Subject-ID
       PCR_Blank            1  Run-ID  Subject-ID
       Water                8  Run-ID  Subject-ID
       human_1             39  Run-ID  Subject-ID
Run3   D6300               14  Run-ID  Subject-ID
       D6305                1  Run-ID  Subject-ID
       D6306                1  Run-ID  Subject-ID
       DZ35298             16  Run-ID  Subject-ID
       DZ35322              5  Run-ID  Subject-ID
       MSA1000              1  Run-ID  Subject-ID
       MSA1001              1  Run-ID  Subject-ID
       MSA1002              1  Run-ID  Subject-ID
       MSA1003              1  Run-ID  Subject-ID
       NTC_Blank            1  Run-ID  Subject-ID
       PCR_Blank            1  Run-ID  Subject-ID
       Water                7  Run-ID  Subject-ID
       human_1             24  Run-ID  Subject-ID
Run4   D6300               21  Run-ID  Subject-ID
       D6305                2  Run-ID  Subject-ID
       D6306                2  Run-ID  Subject-ID
       D6310               22  Run-ID  Subject-ID
       D6311                2  Run-ID  Subject-ID
       DZ35298              7  Run-ID  Subject-ID
       DZ35316              5  Run-ID  Subject-ID
       DZ35322              2  Run-ID  Subject-ID
       MSA1000              2  Run-ID  Subject-ID
       MSA1001              2  Run-ID  Subject-ID
       MSA1002              2  Run-ID  Subject-ID
       MSA1003              2  Run-ID  Subject-ID
       NTC_Blank            2  Run-ID  Subject-ID
       PCR_Blank            2  Run-ID  Subject-ID
       Water               36  Run-ID  Subject-ID
       human_1             40  Run-ID  Subject-ID

<h3 id="2.4&nbsp;&nbsp;Output-manifest">2.4&nbsp;&nbsp;Output manifest</h3>

Sample ID's need to be renamed to include "-", to match the FastQ folder paths. The cleaned manifest was saved used for sequencing.

In [17]:
%cd {proj_dir}

C:\Users\slsevilla\Google Drive\MyDocuments_Current\Education\George Mason University\Dissertation\Data\Aim1


In [18]:
m_nonbio.reset_index(inplace = True)
m_nonbio['Sample ID'].str.replace('_','-')

m_nonbio.to_csv(r'manifest/m_rec.txt',header = m_nonbio.columns,index = False, sep='\t')
m_nonbio.to_csv(r'manifest/m_rec_rm.txt',header = m_nonbio.columns,index = False, sep='\t')
m_nonbio.set_index('Sample ID',inplace=True)

m_save=m_nonbio.copy()
m_save.shape

(389, 14)

<h2 id="3&nbsp;&nbsp;Sequencing-information">3&nbsp;&nbsp;Sequencing information</h2>

<h3 id="3.1&nbsp;&nbsp;Yaml-parameters">3.1&nbsp;&nbsp;Yaml parameters</h3>

All project samples were processed using the following parameters:

In [ ]:
%cd manifest
!cat *.yml

<h3 id="3.2&nbsp;&nbsp;Pipeline-run">3.2&nbsp;&nbsp;Pipeline run</h3>

In [ ]:
#qlogin -q interactive.q
#cd /DCEG/Projects/Microbiome/Analysis/NP0084_MB/20200410_2019.1
#sh run_pipeline.sh

<h2 id="4&nbsp;&nbsp;Quality-control">4&nbsp;&nbsp;Quality control</h2>

<h3 id="4.1&nbsp;&nbsp;Trimming">4.1&nbsp;&nbsp;Trimming</h3>

The trimming parameters for the initial pipeline run (`<datestamp>_initial_run`) are set to 0 (no trimming).  For subsequent runs, trimming parameters are set based on the read quality plots (not shown here; please browse `import_and_demultiplex/<runID>.qzv` using [QIIME's viewer](https://view.qiime2.org/) for quality plots).  For this run, trimming parameters (also found in the config) are as follows:

In [ ]:
%cd {proj_dir}manifest
!grep -A4 "dada2_denoise" *.yml

After error correction, chimera removal, and removal of phiX sequences, the remaining reads are used for taxonomic classification.  We are performing classification with a naive Bayes classifier trained on the SILVA 99% OTUs database that includes only the V4 region (defined by the 515F/806R primer pair).  This data is located at `taxonomic_classification/barplots_classify-sklearn_silva-132-99-515-806-nb-classifier.qzv`.  Please use [QIIME's viewer](https://view.qiime2.org/) for a more detailed interactive plot.

<h3 id="4.2&nbsp;&nbsp;Sequence depth">4.2&nbsp;&nbsp;Sequence depth</h3>
Per-sample read depths are recorded in `import_and_demultiplex/<runID>.qzv`. Distributions per flow cell should be similar if the flow cells contained the same number of non-blank samples. If a flow cell contains fewer samples, each sample will have a greater number of reads, so that the total number of reads produced per flow cell remains approximately the same. The plots below show the "level 1" taxonomic classification, the kingdom level. Absolute abundance is displayed for all samples, as is the relative abundance.

In [ ]:
%%bash
#for i in *qzv; do unzip -q $i -d "rpt_${i%.*}"; done
#for i in rpt_*/*/data/metadata.tsv; do dos2unix -q $i; j=${i%%/*}; k=${j#"rpt_"}; awk -v var="$k" 'BEGIN{FS=OFS="\t"}NR>2{print $0,var}' $i >> t; done
#cat <(echo -e "sample-id\tinput\tfiltered\tdenoised\tmerged\tnon-chimeric\tflow_cell") t > rpt_denoising_stats.tsv
#rm t

In [ ]:
#alternative way to look at the read counts - will be the same as the 'input' col above
#%%bash
#for i in *qzv; do unzip -q $i -d "rpt_${i%.*}"; done
#for i in rpt_*/*/data/per-sample-fastq-counts.csv; do j=${i%%/*}; k=${j#"rpt_"}; awk -v var="$k" 'BEGIN{FS=",";OFS="\t"}$1!~/Sample name/{print $1,$2,var}' $i >> t; done
#cat <(echo -e "Sample_name\tSequence_count\tRun_ID") t > rpt_vertical_per-sample-fastq-counts.csv
#rm t

In [ ]:
#alternative way to look at the read counts - will be the same as the 'input' col above
#df_depth = pd.read_csv('output/import_and_demultiplex/rpt_vertical_per-sample-fastq-counts.csv',sep='\t')
#df_depth.set_index('Sample_name',inplace=True)
#samp_list = m_save.index

#df_depth_clean=df_depth[df_depth.index.isin(samp_list)==True]
#m_save['OTU-count']=df_depth_clean['Sequence_count']
#m_save.head()

In [ ]:
%cd {proj_dir}

In [ ]:
#Read in the denoising stats from QIIME2 output
df_stats = pd.read_csv('output/denoising/stats/rpt_denoising_stats.tsv',sep='\t')
df_stats = df_stats.set_index('sample-id')

df_reads = pd.concat([m_save,df_stats],axis=1)
df_reads.drop(columns='flow_cell',inplace=True) #duplicate column
df_reads = df_reads[df_reads.index.isin(m_save.index)] #only include samples filt above

In [ ]:
def plot_depth(df,label,y_title,file_save):
    plt.figure(dpi=200)
    ax = df.sort_values(0).plot.bar(stacked=True, figsize=(70,7), width=1, edgecolor='white', ax=plt.gca())
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.5),ncol=4,fontsize=52)
    ax.set_ylabel(y_title,fontsize=52)
    ax.axes.get_xaxis().set_ticks([])
    ax.set_title('Sequencing Depth - '+label,fontsize=52)
    ax.set_yticklabels(ax.get_yticks(), size = 40)
    plt.savefig(file_save+'.png',bbox_inches="tight")  
    plt.show()    

In [ ]:
plot_depth(df_reads['input'],"All Samples","Number of reads","output/img/num_reads_all")

In [ ]:
read_stat=[df_reads['input'].mean(),df_reads['input'].min(),df_reads['input'].max()]
read_stat

In [ ]:
col_add = ['input','filtered','denoised','merged','non-chimeric']
for i in col_add:
    m_save[i] = df_reads[i]

In [ ]:
m_save.shape

<h3 id="4.3&nbsp;&nbsp;QIIME-filtering">4.3&nbsp;&nbsp;QIIME filtering</h3>

Per-sample read depths at each filtering step are recorded in `denoising/stats/<runID>.qzv`.  The plots below show the mean for each category; error bars indicate the 95% confidence interval.  

These filtering steps include:

- filter: sequences are filtered based on Q score (default is 4)
- denoise: 
- unpaired: paired-end sequences that are unable to be merged, either because the sequences are too short and do not overlap, or because the ends do not align;
- non-chimeric: sequences that are chimeric are fitered;

Blanks (ext_blanks, seq_blanks) should have signficantly lower reads than artifical colonies (art_col, Zymobiomics, MSA), which would both have lower reads than study samples (human_1, biocollective, robogut). Additionally, blanks were sub-divided further, as NTC blanks are expected to have near-zero read depths, and represent false positives introduced by sequencing reagents.
PCR blanks and Extraction blanks are expected to have read depths that are at least one to two orders of magnitude lower than the average study sample depth.  They represent the relatively low level of taxa that may be detected in the water, or other reagents used in the lab.

In [ ]:
#determine which samples have 0 reads at start
if(len(m_save[m_save['input']==0])!=0):
    metacount_all(m_save[m_save['input']==0])
else:
    print("No samples meet metric")

In [ ]:
#determine which samples have 0 reads after filtering
if(len(m_save[m_save['filtered']==0])!=0):
    metacount_all(m_save[m_save['filtered']==0])
else:
    print("No samples meet metric")

In [ ]:
#determine which samples have 0 reads after denoising
rm = m_save[(m_save['denoised']==0) & (m_save['filtered']!=0)]
if(len(rm)):
    metacount_all(rm)
else:
    print("No samples meet metric")

In [ ]:
#determine which samples have 0 reads after unpaired
rm = m_save[(m_save['merged']==0) &(m_save['denoised']!=0) & (m_save['filtered']!=0)]

if(len(rm)):
    metacount_all(rm)
    display(rm.shape)
else:
    print("No samples meet metric")

In [ ]:
#determine which samples have 0 reads after chimeria
rm = m_save[(m_save['non-chimeric']==0) & (m_save['merged']!=0) &(m_save['denoised']!=0) & (m_save['filtered']!=0)]
if(len(rm)):
    metacount_all(rm)
    display(rm.shape)
else:
    print("No samples meet metric")

In [ ]:
#remove all samples from df
df_reads_clean = m_save[m_save['non-chimeric']!=0]
df_reads_clean.shape

In [ ]:
m_save = df_reads_clean.copy()

In [ ]:
metacount_all(m_save)

<h4 id="4.3.2&nbsp;&nbsp;Non-bacterial Reads">4.3.2&nbsp;&nbsp;Non-bacterial Reads</h4>
Reads are being classified using a database of predominantly bacterial sequences, so human reads, for example, will generally be in the "Unclassified" category rather than "Eukaryota."  Non-bacterial reads can indicate host (human) or other contamination. This can be helpful with troubleshooting if some samples unexpectedly have a high proportion of non-bacterial OTU's. Samples with > 10% non-bacterial OTU's were filtered.

In [ ]:
%cd {proj_dir}
#!unzip -q -d rpt_silva 'barplots_classify-sklearn_silva-132-99-nb-classifier.qzv'

In [ ]:
def read_in_tax(level):
    f = glob.glob('output/taxonomic_classification/rpt_silva/*/data/level-'+level+'.csv')
    df = pd.read_csv(f[0])
    df = df.rename(columns = {'index':'Sample'})
    df = df.set_index('Sample')  
    return df

In [ ]:
def plot_abund(df,label,y_title,file_save):
    plt.figure(dpi=200)
    pal = sns.color_palette("Accent")
    ax = df.sort_values('D_0__Bacteria').plot.bar(stacked=True, color=pal, figsize=(70,7), width=1, edgecolor='white', ax=plt.gca())
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.5),ncol=4,fontsize=52)
    ax.set_ylabel(y_title,fontsize=52)
    ax.axes.get_xaxis().set_ticks([])
    ax.set_title('Taxonomic classification - '+label,fontsize=52)
    ax.set_yticklabels(ax.get_yticks(), size = 40)
    plt.savefig(file_save+'.png',bbox_inches="tight")  
    plt.show()    

In [ ]:
df_l1 = read_in_tax("1") #Read in taxonomy at kingdom

In [ ]:
#subset df to current samp list
samp_list = m_save.index
df_l1_clean = df_l1[~df_l1.index.isin(samp_list)==False]
df_l1_clean.shape

In [ ]:
df_l1_abund = df_l1_clean.select_dtypes(['number']).dropna(axis=1, how='all')  
plot_abund(df_l1_abund,'All Samples','Num of OTUs','output/img/abs_l1_all')

df_l1_rel = df_l1_abund.div(df_l1_abund.sum(axis=1), axis=0) * 100 #axis=1 is along row
plot_abund(df_l1_rel,'All Samples','Relative frequency (%)','output/img/rel_l1_all')

In [ ]:
df_l1_rel.sort_values('D_0__Bacteria').head(20)

In [ ]:
df_l1_rel[df_l1_rel['D_0__Bacteria']>99].count()

In [ ]:
#remove samps with <99% bact
df_l1_rel_clean = df_l1_rel[df_l1_rel.index.isin(m_save.index)]
if len(df_l1_rel_clean[(df_l1_rel_clean['D_0__Bacteria'] <99)]>0):
    df_l1_drop_list = df_l1_rel_clean[(df_l1_rel['D_0__Bacteria'] <99)].index
    m_bac90_rm = m_save[~m_save.index.isin(df_l1_drop_list) == False]
    m_bac90_rm.to_csv(r'manifest/m_90bact_rm.txt',header = m_bac90_rm.columns,index = False, sep='\t')
    
    m_bac90 = m_save[m_save.index.isin(df_l1_drop_list) == False]
    m_bac90.to_csv(r'manifest/m_90bact.txt',header = m_bac90.columns,index = True, sep='\t')
    display(df_l1_rel_clean[(df_l1_rel_clean['D_0__Bacteria'] <99)])
else:
    print ("all are above 90")
    display(df_l1_rel_clean.sort_values(by=['Unassigned']).tail())

In [ ]:
metacount_all(m_bac90_rm)

In [ ]:
m_bac90.shape

In [ ]:
m_save = m_bac90.copy()
m_save.shape

In [ ]:
m_save.head()

In [ ]:
king_list = ['D_0__Archaea','D_0__Bacteria','D_0__Eukaryota','Unassigned']
for i in king_list:
    m_save[i] = df_l1[i]


<h4 id="4.3.3&nbsp;&nbsp;By flowcell">4.3.3&nbsp;&nbsp;By flowcell</h4>

In [ ]:
#Plot fig with all samples
def plot_depth(df_in):
    plt.figure(dpi=100)
    sns.set(style="whitegrid")
    ax = sns.boxplot(x="Run-ID",y="input",data=df_in)
    ax.set_xlabel('Flowcell ID')
    ax.set_xticklabels(ax.get_xticklabels(),rotation=40,ha="right")#,fontsize=8)
    ax.axes.set_title("Sequencing depth distribution per flow cell - All samples",fontsize=12)
    plt.savefig('output/img/seq_depth_uneven.png',bbox_inches="tight")  
    plt.show()

In [ ]:
plot_depth(m_save)

In [ ]:
metacount_run_sub(m_save) #Check if blanks are even amongst all samples

In [ ]:
#Remove blanks, re-run and see if there is sig
search_values = ['Ext_Control','Seq_Control']
df_blanks = m_save[m_save['Sample-Type'].str.contains('|'.join(search_values ))]
drop_list = df_blanks.index

df_depth_no_blanks = m_save[~m_save.index.isin(drop_list) == False]
plot_depth(df_depth_no_blanks)

<h4 id="4.3.3&nbsp;&nbsp;By sample type">4.3.3&nbsp;&nbsp;By sample type</h4>

In [ ]:
col_list = ['D_0__Archaea','D_0__Bacteria','D_0__Eukaryota','Unassigned']

for i in m_save['Sample-Type'].unique():
    drop_list = m_save[m_save['Sample-Type']==i].index
    df = df_l1[df_l1.index.isin(drop_list)==True]
    df = df[col_list].dropna(axis=1, how='all')  
    plot_abund(df,i,'Number of OTUs','output/img/abs_l1_'+i)
    df2 = df.div(df.sum(axis=1), axis=0) * 100 #axis=1 is along row
    plot_abund(df2,i,'Relative frequency (%)','output/img/rel_l1_'+i)

<h3 id="4.4&nbsp;&nbsp;Statistical differences">4.4&nbsp;&nbsp;Statistical differences</h3>

The Mann-Whitney U Test is a null hypothesis test, used to detect differences between two independent data sets. The test is specifically for non-parametric distributions, which do not assume a specific distribution for a set of data. Because of this, the Mann-Whitney U Test can be applied to any distribution, whether it is Gaussian or not.

Specifically, the null hypothesis of the Mann-Whitney U Test states that the distributions of two data sets are identical. If the null hypothesis is correct, there is a 50 percent chance that an arbitrarily selected value in one distribution is greater than another arbitrarily selected value in the second distribution (2).

The first output, the u-statistic, is defined in the previous section as the test statistic U for the Mann-Whitney U Test. The U-statistic is interpreted using a two-tailed test table, where the table contains critical values of U. To reject the null hypothesis at α=0.05, the U obtained from the test must be below the critical value of U found in the test table.

In [ ]:
def mann_whitney_u_test(des1, des2, distribution_1, distribution_2):
    u_statistic, p_value = stats.mannwhitneyu(distribution_1, distribution_2, alternative='two-sided')
    pair = des1 + " vs " + des2
    return pair, u_statistic, p_value

In [ ]:
def mann_whitt_sig(df_in,level,y_in):
    des_list = list(df_in[level].unique()) #create list of unique elements
    pair_list = list(combinations(des_list,2)) #create pair list for len of element list
    
    tmp = []
    for i in range(0,len(pair_list)): 
        des1 = pair_list[i][0]
        des2 = pair_list[i][1]
        tmp.append(mann_whitney_u_test(des1, des2, list(df_in[(df_in[level] == des1)][y_in]), list(df_in[(df_in[level] == des2)][y_in])))
    
    df = pd.DataFrame(tmp)
    df['sig'] = "nonsig"
    
    for i in df.index:
        if df.iloc[i][2] < 0.0001:
            df.at[i,'sig']= 'sig'
    df.columns = ["pair","u_stat","p_values","sig_.0001"]
    return df

<h4 id="4.4.1&nbsp;&nbsp;By sequence reads">4.4.1&nbsp;&nbsp;By sequence reads</h4>

In [ ]:
df_mannwhit_stat = mann_whitt_sig(df_depth_no_blanks,'Run-ID','input')
print(df_mannwhit_stat)

In [ ]:
def mann_whitt_plot_sig(df_in,x_filt_in,y_filt_in,order_in,pairs_in,filename):
    sns.set(style="whitegrid")
    df = sns.load_dataset("tips")

    x = x_filt_in
    y = y_filt_in
    order = order_in
    ax = sns.boxplot(data=df_in, x=x, y=y, order=order)
    add_stat_annotation(ax, data=df_in, x=x, y=y, order=order,
                        box_pairs=pairs_in,
                        test='Mann-Whitney', text_format='star', loc='outside', verbose=2)
    plt.savefig('output/img/'+filename+'.png', dpi=300, bbox_inches='tight')

In [ ]:
#Plot study samples for sig
list_order = list(df_depth_no_blanks["Run-ID"].unique()) #create list of unique elements
list_pairs = list(combinations(list_order,2)) #create pair list for len of element list

x_filt = "Run-ID"
y_filt = "input"
mann_whitt_plot_sig(df_depth_no_blanks,x_filt,y_filt,list_order,list_pairs,'mannwhitt_seqcount_study')

<h4 id="4.4.2&nbsp;&nbsp;By qiime2 filtering">4.4.1&nbsp;&nbsp;By qiime2 filtering</h4>

In [ ]:
#Plots read counts for each level of filtering phases
def plot_read_counts(samples,pop):
    plt.figure(dpi=100)
    sns.set(style="whitegrid")
    ax = sns.barplot(data=df_stats[df_stats.index.isin(samples)]).set_title('Number of reads in ' + pop + ' samples')
    plt.savefig('output/img/filt_values_'+pop+'.png',bbox_inches="tight")  
    plt.show()

In [ ]:
#subsets df by filter and calls plot_read_counts
def subplot_read_counts(df_in,sub_in):
    for i in df_in[sub_in].unique():
        l = list(df_in[df_in[sub_in].str.match(i)].index)
        plot_read_counts(l,i)

In [ ]:
subplot_read_counts(m_save,'Sample-Type')
subplot_read_counts(m_save,'Sample-Des')

In [ ]:
#Create a fraction of filtered reads col
m_save['frac_filtered'] = ((m_save['input'] - m_save['non-chimeric']) / m_save['input'])*100

In [ ]:
#Remove blanks and see if there is sig in study samples
search_values = ['Ext_Control','Seq_Control']
df_blanks = m_save[m_save['Sample-Type'].str.contains('|'.join(search_values ))]
drop_list = df_blanks.index

df_no_blanks = m_save[~m_save.index.isin(drop_list) == False]

In [ ]:
#Plot study samples for sig
list_order = list(df_depth_no_blanks["Run-ID"].unique()) #create list of unique elements
list_pairs = list(combinations(list_order,2)) #create pair list for len of element list

x_filt = "Run-ID"
y_filt = "frac_filtered"
mann_whitt_plot_sig(df_no_blanks,x_filt,y_filt,list_order,list_pairs,'mannwhitt_fracfilt_study')

In [ ]:
#Remove study samples and see if there is sig in controls
search_values = ['Ext_Control','Seq_Control']
df_controls = m_save[~m_save['Sample-Type'].str.contains('|'.join(search_values ))]
drop_list = df_controls.index

df_no_study = m_save[~m_save.index.isin(drop_list) == True]

In [ ]:
#Plot study samples for sig
list_order = list(df_depth_no_blanks["Run-ID"].unique()) #create list of unique elements
list_pairs = list(combinations(list_order,2)) #create pair list for len of element list

x_filt = "Run-ID"
y_filt = "frac_filtered"
mann_whitt_plot_sig(df_no_study,x_filt,y_filt,list_order,list_pairs,'mannwhitt_fracfilt_controls_run')

In [ ]:
#Plot study samples for sig
list_order = list(df_depth_no_blanks["Sample-Type"].unique()) #create list of unique elements
list_pairs = list(combinations(list_order,2)) #create pair list for len of element list

x_filt = "Sample-Type"
y_filt = "frac_filtered"
mann_whitt_plot_sig(df_no_study,x_filt,y_filt,list_order,list_pairs,'mannwhitt_fracfilt_controls_st')

In [ ]:
#plot clean data
col_list = ['D_0__Archaea','D_0__Bacteria','D_0__Eukaryota','Unassigned']

df_l1_abund = m_save[col_list].dropna(axis=1, how='all')  
plot_abund(df_l1_abund,'All Samples','Num of OTUs','output/img/abs_l1_clean')

df_l1_rel = df_l1_abund.div(df_l1_abund.sum(axis=1), axis=0) * 100 #axis=1 is along row
plot_abund(df_l1_rel,'All Samples','Relative frequency (%)','output/img/rel_l1_clean')

In [ ]:
m_save.to_csv(r'manifest/m_cleaned.txt',header = m_save.columns,index = True, sep='\t')

Taxonomy

In [ ]:
tax_complete = pd.read_csv(proj_dir+'taxonomy/Taxonomy_Complete.csv')

In [ ]:
tax_complete

In [ ]:
tax = tax_complete.groupby('Genus').sum()
tax_genus = pd.DataFrame(tax)
pd.options.display.float_format = '{:.2E}'.format
tax_genus

In [ ]:
tax_genus.to_csv(r'taxonomy/tax_genus.txt',header = tax_genus.columns,index = True, sep='\t')